### Best-First Search in the Blocksworld###

Goals of this lab
1. Some Python programming
1. More work with Jupyter notebooks
1. A search example and code that's like the Pacman search assignment but not exactly
1. Experience with a code base that's part of Assignment 1
1. Show the various components of a search problem broken up into classes/modules (the Pacman code does not do that so well)
1. Blocksworld-like problems have been an important part of AI research since the beginning, and for good reason

------------------------------------------------

#### The Blocksworld Scenario ####

* There are two surfaces, the *table* and the *stack*
  * Blocks on the *table* are not stacked on each other
  * There is exactly one stack of 0 or more blocks on the *stack*
* There is a single gripper, which 
  * Can hold up to one block
  * Is located either at the *table* or at the *stack*
* Every block is either on the table, on the stack, or being held by the gripper
  
The operators available are

* **pickup(block)**
  * Precondition
    * The gripper must be empty
    * The argument must be the name of a block
    * The block must be at the same location as the gripper
    * The block must not have another block on top of it
  * Postcondition
    * The gripper is holding the block
    * The block is no longer at its previous position

* **putdown(block)**
  * Precondition
    * The gripper must be holding a block
    * The argument must be the name of a block
  * Postcondition
    * The gripper is empty
    * The block is at the location of the gripper
      * If the location is the *stack*, the block is at the top of the stack

* **move(surface)**
  * Precondition
    * The argument must be the name of a surface
  * Postcondition
    * The gripper is located at that surface


---------------------------------------------------

****Initial and Goal States****

The system should be flexible on handling various initial states and goals, but for this class we will consider two problems, one easy and one hard.  (Why would the hard one be hard?)

**Easy Problem** ![Easy Problem](blocksWorldEasy.GIF)
**Difficult Problem** ![Difficult Problem](blocksWorldDifficult.GIF)


### Problem-Independent Machinery

In [ ]:
## The search algorithm itself -- it takes a problem, which will give it an initial state and the goal test,
##   the world state itself which gives it the successor states, and an evaluator that evaluates the quality
##   of a node on the search frontier

#  Priority queue code taken from Pacman project -- PriorityQueue supports
#      pop, isEmpty, and push/update
#
#  Client supplies
#    -- a WorldState; a WorldState implements the method successors()
#    -- a Problem which supplies the initial state and goal state checker
#    -- an Evaluator which supplies a method that evaluates a WorldState
#
#   The search function uses a SearchState which is a WorldState plus a sequence of 
#     actions (not examined by Search).   The search fringe is a priority 
#     queue of SearchState
#
#   Search returns a 2-tuple -- 
#    -- a sequence of actions
#    -- performance information:  process time used, number of nodes expanded, 
#         number of nodes skipped (because they were previously expanded)

from priorityqueue import PriorityQueue
import time

def aStarSearch(problem, evaluator, verbose=None):
    startTime = time.process_time()
    fringe = PriorityQueue()
    visited = {}
    initialWorldState = problem.initial()
    initialValue = evaluator.value(initialWorldState, [])
    initialSearchState = SearchState(initialWorldState, [])
    fringe.update(initialSearchState, initialValue)
    numVisited = numSkipped = 0
    while (True):
        if fringe.isEmpty():
            return (None, (time.process_time() - startTime, numVisited, numSkipped))
        nextNode = fringe.pop()   # A search state (state, actions)
        numVisited += 1
        if (verbose and numVisited % verbose == 0):
            print("Visited " + str(numVisited) + " world is " + str(nextNode._worldState))
            print("Skipped " + str(numSkipped) + " Fringe is size " + str(len(fringe.heap)))
            print("Evaluation is " + str(evaluator.value(nextNode._worldState, nextNode._actions)) + " with actions " + str(len(nextNode._actions)))
        if (problem.isGoal(nextNode.worldState())):
            return (nextNode._actions, (time.process_time() - startTime, numVisited, numSkipped))
        if (nextNode._worldState in visited):
            numSkipped += 1
        else:
            visited[nextNode.worldState()] = True
            successors = nextNode.worldState().successors()
            for successor in successors:
                state, action = successor
                actions = list(nextNode.actions())
                actions.append(action)
                newSS = SearchState(state, actions)
                newValue = evaluator.value(state, actions)
                fringe.update(newSS, newValue)
    raise "Impossible search execution path."

## Instances of SearchState go on the search fringe -- contains both a state and 
## list of actions so far

class SearchState:
    def __str__(self):
        return "{S " + str(self._worldState) + "/" + str(self._actions) + "}"
    
    def __init__(self, worldState, actions):
        self._worldState = worldState
        self._actions = actions
    
    def worldState(self):
        return self._worldState
    
    def actions(self):
        return self._actions

### Interface Between Search and Client ###

In [ ]:
class WorldState:
    #  Method successors() returns a tuple:  (worldState, action)
    def successors():
        raise "Not implemented"

class Problem:
    def __init__(self, initialSpec, goalSpec):
        self.initialSpec = initialSpec
        self.goalSpec = goalSpec
    
    # Method initial returns a world state
    def initial(self):
        raise "Not implemented"
        
    # Method isGoal returns a boolean
    def isGoal(self, state):
        raise "Not implemented"

# Evaluator provides the evaluator f(s) = g(s) + h*(s)

class Evaluator:
    def __init__(self, goalEstimator, actionsCoster):
        self._estimator = goalEstimator
        self._coster = actionsCoster
    def estimateToGoal(self, state):
        return self._estimator(state)
    def costSoFar(self, actions):
        return self._coster(actions)
    def value(self, state, actions):
        return self.estimateToGoal(state) + self.costSoFar(actions)
    

### Domain-Specific Code ###

****The World State Description****

We need some data structure that holds the state of the world at some point in time.  It is important that this representation be compact because there will be many copies in the search queue.  For now we will be moderately thrifty with space.  Remember we need to capture
* What the gripper is holding
* Where the gripper is
* The blocks on the table (in any order)
* The blocks on the stack (in stacked order)

The world state class also has the operator definitions -- so it describes the "physics" of the blocks world.

In [ ]:
import copy

class BlocksWorldState(WorldState):

    def __str__(self):
        return "{" + str(self._table) + "|" + str(self._stack) + "|" + str(self._gripperHolding) + "|"\
            + self._gripperLocation + "}"
    
    #  Any world state needs to redefine equality and hash so instances go on the priority queue correctly
    #  and the duplication check works
    
    def __eq__(self, other):
        if isinstance(other, BlocksWorldState):
            return self._table == other._table and self._stack == other._stack \
                and self._gripperHolding == other._gripperHolding and self._gripperLocation == other._gripperLocation
        else:
            return False

    def __hash__(self):
        return hash(str(self._table) + str(len(self._table)) + str(self._gripperHolding) + str(self._stack) + \
                                           str(len(self._stack)) + str(self._gripperLocation))
    
    # NB: every successor state must deep copy the old state!
    
    def successors(self):
        candidates = [pickup(self), putdown(self), move(self)]
        candidates = [val for sublist in candidates for val in sublist]    # list flatten
        return candidates
    
    # Preconsition:  must not be holding anything
    # Effect:  if gripper is at the table, can pick up any block on the table, so return a list
    #          if gripper is at the stack, can pick up the top block of the stack, _stack.pop()
    def pickup(self):
        # return [(s, "pickup" + block)...]
 
    # Precondition:  must be holding something
    # Effect:  if gripper is at the table, block is on the table, and gripper is holding nothing
    #          if gripper is at the stack, put the block at the top of the stack _stack.append, and gripper is holding nothing
    
   def putdown(self):
        if (not self._gripperHolding):
            return []
        elif (self._gripperLocation == "T"):
            s = copy.deepcopy(self)
            block = s._gripperHolding
            s._table.append(block)
            s._gripperHolding = None
            return [(s, "putdown" + block)]
        elif (self._gripperLocation == "S"):
            s = copy.deepcopy(self)
            block = s._gripperHolding
            s._stack.append(block)
            s._gripperHolding = None
            return [(s, "putdown" + block)]
        else:
            raise "Unknown gripper location " + _gripperLocation
 
      # return [(s, "putdown" + block)]
            
    # Precondition:  none
    # Effect:  gripperLocation changes from "S" to "T" and back
    
    def move(self):
        s = copy.deepcopy(self)
        s._gripperLocation = "S" if s._gripperLocation == "T" else "T"
        return [(s, "move" + s._gripperLocation)]   


----------------------------------------------

In [ ]:
## Blocks World Problem will take two inputs
##   -- initialSpec is a dictionary with keys "table" and "stack" each with a list of strings, 
##        gripperHolding and gripperLocation, each a string either "T" table or "S" stack
##   -- goalSpec is a dictionary with key "stack" and a list of strings;  so this Problem's
##        goal only cares about what is on the stack

class BlocksWorldProblem(Problem):
    def __init__(self, initialSpec, goalSpec):
        super(BlocksWorldProblem, self).__init__(initialSpec, goalSpec)
  
    def initial(self):
        state = BlocksWorldState()
        initspec = self.initialSpec
        state._table = initspec["table"]
        state._stack = initspec["stack"]
        state._gripperHolding = initspec["gripperHolding"]
        state._gripperLocation = initspec["gripperLocation"]
        return state
    
    def isGoal(self, state):
        return state._stack == self.goalSpec["stack"]


In [ ]:
#  Explore different estimator functions here

def goalEstimator(state, goalStack):
    return 0


In [ ]:
## This is the full definition of a search problem -- an initial, a goal state, and an evaluator

easyProblemGoalStack = ["D", "C"]
easyBlocksWorldProblem = BlocksWorldProblem(\
    {"table": ["A", "C", "D", "E"], "stack": ["B"], "gripperHolding": None, "gripperLocation": "T"},\
    {"stack": easyProblemGoalStack})

easyBlocksWorldEvaluator = Evaluator(lambda state: 0, lambda actions: len(actions))

In [ ]:
actions, stats = aStarSearch(easyBlocksWorldProblem, easyBlocksWorldEvaluator, verbose=100)
print(actions)
print(stats)

In [ ]:
## This is the hard problem as pictured above

hardProblemGoalStack = ["A", "B", "C"]
hardBlocksWorldProblem = BlocksWorldProblem(\
    {"table": ["E", "F", "G", "H"], "stack": ["C", "B", "A"], "gripperHolding": None, "gripperLocation": "T"},\
    {"stack": hardProblemGoalStack})

hardBlocksWorldEvaluator = Evaluator(lambda state: goalEstimator(state, hardProblemGoalStack), uniformCostCoster)

In [ ]:
actions, stats = aStarSearch(hardBlocksWorldProblem, hardBlocksWorldEvaluator, verbose=1000)
print(actions)
print(stats)